In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler
import pybithumb
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)

Scaler = MinMaxScaler()


dir = 'ohlcv/'
ohlcv_list = os.listdir(dir)


def min_max_scaler(price):
    Scaler = MinMaxScaler()
    Scaler.fit(price)

    return Scaler.transform(price)


def low_high(Coin, input_data_length, trade_limit=None):

    #   거래 제한은 고점과 저점을 분리한다.

    #   User-Agent Configuration
    ohlcv_excel = pybithumb.get_ohlcv(Coin, 'KRW', 'minute1')

    price_gap = ohlcv_excel.close.max() / ohlcv_excel.close.min()
    if (price_gap < 1.07) and (trade_limit is not None):
        return None, None

    obv = [0] * len(ohlcv_excel)
    for m in range(1, len(ohlcv_excel)):
        if ohlcv_excel['close'].iloc[m] > ohlcv_excel['close'].iloc[m - 1]:
            obv[m] = obv[m - 1] + ohlcv_excel['volume'].iloc[m]
        elif ohlcv_excel['close'].iloc[m] == ohlcv_excel['close'].iloc[m - 1]:
            obv[m] = obv[m - 1]
        else:
            obv[m] = obv[m - 1] - ohlcv_excel['volume'].iloc[m]
    ohlcv_excel['OBV'] = obv

    ohlcv_excel['MA60'] = ohlcv_excel['close'].rolling(60).mean()

    closeprice = ohlcv_excel['close'].iloc[-1]

    # ----------- dataX, dataY 추출하기 -----------#
    #   OBV :
    ohlcv_data = ohlcv_excel.values[1:].astype(np.float)

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #          데이터 전처리         #
        #   Fixed X_data    #
        price = ohlcv_data[:, :4]
        volume = ohlcv_data[:, [4]]
        OBV = ohlcv_data[:, [-2]]
        MA60 = ohlcv_data[:, [-1]]

        scaled_price = min_max_scaler(price)
        scaled_volume = min_max_scaler(volume)
        scaled_OBV = min_max_scaler(OBV)
        scaled_MA60 = min_max_scaler(MA60)
        # print(scaled_MA60.shape)

        x = np.concatenate((scaled_price, scaled_volume, scaled_MA60), axis=1)  # axis=1, 세로로 합친다

        if (x[-1][1] > 0.3) and (trade_limit is not None):
            return None, None

        # print(x.shape)  # (258, 6)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        for i in range(input_data_length, len(ohlcv_data) + 1):  # 마지막 데이터까지 다 긇어모은다.
            group_x = x[i - input_data_length:i]
            dataX.append(group_x)  # dataX 리스트에 추가

        if (len(dataX) < 100) and (trade_limit is not None):
            return None, None

        X_test = np.array(dataX)
        row = X_test.shape[1]
        col = X_test.shape[2]

        X_test = X_test.astype('float32').reshape(-1, row, col, 1)

        return X_test, closeprice


def made_x(file, input_data_length, model_num, check_span, Range_fluc, get_fig):

    ohlcv_excel = pd.read_excel(dir + file, index_col=0)

    # 이후 check_span 개 데이터의 고 / 저점의 폭
    ohlcv_excel['fluc_close'] = ohlcv_excel['close'].shift(-check_span).rolling(check_span).max() / \
                                ohlcv_excel['close'].shift(-check_span).rolling(check_span).min()
    ohlcv_excel['fluc_close'] = np.where(ohlcv_excel['fluc_close'] > Range_fluc, 1, 0)

    # ----------- dataX, dataY 추출하기 -----------#
    # print(ohlcv_excel.tail(20))
    # ohlcv_excel.to_excel('test.xlsx')
    # quit()

    # NaN 제외하고 데이터 자르기 (데이터가 PIXEL 로 들어간다고 생각하면 된다)
    # MA60 부터 FLUC_CLOSE, 존재하는 값만 슬라이싱
    # ohlcv_data = ohlcv_excel.values[ohlcv_excel['MA60'].isnull().sum(): -check_span].astype(np.float)
    ohlcv_data = ohlcv_excel.values[:-check_span].astype(np.float)
    # print(len(ohlcv_data))
    # print(list(map(float, ohlcv_data[0])))
    # quit()

    # 결측 데이터 제외
    if len(ohlcv_data) != 0:

        #           데이터 전처리         #
        #       x data      #
        price = ohlcv_data[:, :4]
        # volume = ohlcv_data[:, [4]]
        # OBV = ohlcv_data[:, [-3]]
        # MA60 = ohlcv_data[:, [-2]]

        #       y data      #
        fluc_close = ohlcv_data[:, [-1]]

        scaled_price = min_max_scaler(price)
        # scaled_volume = min_max_scaler(volume)
        # scaled_OBV = min_max_scaler(OBV)
        # scaled_MA60 = min_max_scaler(MA60)
        # print(scaled_MA60.shape)

        # print(fluc_close.shape)

        # x = np.concatenate((scaled_price, scaled_volume, scaled_OBV), axis=1)  # axis=1, 세로로 합친다
        x = scaled_price
        y = fluc_close

        # print(x.shape, y.shape)  # (258, 6) (258, 1)
        # quit()

        dataX = []  # input_data length 만큼 담을 dataX 그릇
        dataY = []  # Target 을 담을 그릇

        for i in range(input_data_length, len(y)):
            # group_x >> 이전 완성된 데이터를 사용해보도록 한다. (진입하는 시점은 데이터가 완성되어있지 않으니까)
            group_x = x[i - input_data_length: i]  # 0 부터 len(y) - 2 까지 대입
            group_y = y[i]
            # print(group_x.shape)  # (28, 6)
            # print(group_y.shape)  # (1,)
            # quit()
            # if i is input_data_length + 1:
            #     print(group_x, "->", group_y)
            #     quit()
            dataX.append(group_x)  # dataX 리스트에 추가
            dataY.append(group_y)  # dataY 리스트에 추가

        sliced_ohlcv = ohlcv_data[input_data_length:, :4]

        # ----------- FLUC_CLOSE TO SPAN, 넘겨주기 위해서 INDEX 를 담아주어야 한다. -----------#
        if get_fig == 1:
            spanlist = []
            for m in range(len(fluc_close)):
                if fluc_close[m] > 0.5:
                    if m + 1 < len(fluc_close):
                        spanlist.append((m, m + 1))
                    else:
                        spanlist.append((m - 1, m))

            # ----------- 인덱스 초기화 됨 -----------#

            # ----------- Chart 그리기 -----------#
            plt.plot(min_max_scaler(ohlcv_data[:, [1]]), 'gold', label='close')
            # plt.plot(scaled_OBV, 'b', label='OBV')
            plt.legend(loc='upper right')

            # Spanning
            for i in range(len(spanlist)):
                plt.axvspan(spanlist[i][0], spanlist[i][1], facecolor='c', alpha=0.7)

            Date = file.split()[0]
            Coin = file.split()[1].split('.')[0]
            plt.savefig('./Figure_data/%s_%s/%s %s.png' % (input_data_length, model_num, Date, Coin), dpi=500)
            plt.close()
            # plt.show()
            # ----------- Chart 그리기 -----------#

        return dataX, dataY, sliced_ohlcv


if __name__ == '__main__':

    #           Params          #
    input_data_length = 200
    model_num = input('Press model num : ')
    check_span = 30
    Range_fluc = 1.07  # >> Best Param 을 찾도록 한다.
    get_fig = 1

    #       Make folder      #
    try:
        os.mkdir('./Figure_data/%s_%s/' % (input_data_length, model_num))

    except Exception as e:
        pass

    Made_X = []
    Made_Y = []

    for file in ohlcv_list:

        if int(file.split()[0].split('-')[1]) != 1:
            continue

        result = made_x(file, input_data_length, model_num, check_span, Range_fluc, get_fig)
    
        # ------------ 데이터가 있으면 dataX, dataY 병합하기 ------------#
        if result is not None:

            Made_X += result[0]
            Made_Y += result[1]

            # 누적 데이터량 표시
            print(file, len(Made_X))  # 현재까지 321927개
            # if len(Made_X) > 100000:
            # quit()

    # SAVING X, Y
    X = np.array(Made_X)
    Y = np.array(Made_Y)

    np.save('./Made_X/Made_X %s_%s' % (input_data_length, model_num), X)
    np.save('./Made_X/Made_Y %s_%s' % (input_data_length, model_num), Y)
